In [1]:
import pandas as pd 
import numpy as np
from numpy.random import seed
import tensorflow as tf
import random as rn
seed_value=2509
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
rn.seed(seed_value)
from sklearn import preprocessing                            
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split 
import matplotlib.pyplot as plt
from keras import models
import seaborn as sns

c:\Users\Hippolyte\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Hippolyte\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


ModuleNotFoundError: No module named 'tensorflow'

### Loading the datasets

In [ ]:
train = pd.read_csv('/data/train.csv')
test = pd.read_csv('/data/test.csv')

In [ ]:
train.head()

In [ ]:
combo = [train, test]
for item in combo:
    item['Sex'] = item['Sex'].map( {'male': 1, 'female': 0} ).astype(int)

In [ ]:
for item in combo:
    item['FamilySize'] = item['SibSp'] + item['Parch']+1 
 

In [ ]:
# Let's examine how the size of a family affected survival , size of 1 means travelling alone
train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean()

In [ ]:
for item in combo:
    item.loc[(item['FamilySize'] ==1) , 'Fsize'] = 0
    item.loc[(item['FamilySize'] ==2) , 'Fsize'] = 1
    item.loc[(item['FamilySize'] ==3) , 'Fsize'] = 2
    item.loc[(item['FamilySize'] ==4) , 'Fsize'] = 3
    item.loc[(item['FamilySize'] >4) , 'Fsize'] = 4
    item['Fsize']=item['Fsize'].astype(int)

In [ ]:
for item in combo:
    item['Title'] = item.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # We do not need Name feature anymore
    item = item.drop(['Name'], axis=1, inplace=True)

print("Titles in test dataset")
print(' ')
print(test.Title.unique())
print(' ')
print("Titles in train dataset")
print(' ')
print(train.Title.unique())

In [ ]:
for item in combo:
    item['Title'] = item['Title'].replace(['Ms','Lady', 'Countess','Dona'],'Mrs')
    item['Title'] = item['Title'].replace(['Mme','Mlle'], 'Miss')
    item['Title'] = item['Title'].replace(['Major', 'Sir', 'Jonkheer', 'Dr','Col','Don', 'Capt','Rev'],'Mr')

In [ ]:
#Oups , passenger 797 is a woman titled Dr. so we have to change category to 'Mrs'
print(train.loc[(train.PassengerId== 797) ])

In [ ]:
print("#############  Test dataset titles ###############")
print(train.Title.value_counts())
print("#############  Train dataset titles ###############")
print(test.Title.value_counts())

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#Let's extract the first letters from cabins
for item in combo:
    item['Cabin'].fillna('N', inplace=True)
    item['Cabin'] = item['Cabin'].map(lambda c: c[0])

In [ ]:
# If we examine all first letters from both sets we see a type T,it is category A as we will analyse below, so we replace it.
print('Train set Cabin names',train.Cabin.unique())
print('Test set Cabin names',test.Cabin.unique())
train['Cabin'] = train['Cabin'].replace(['T'], 'A')

In [ ]:
sns.barplot(x="Cabin", y="Survived", data=train)
plt.show()

In [ ]:
for item in combo:
    item['Cabin'] = item['Cabin'].replace(['A', 'B','C',"D",'E','T'], 0)
    item['Cabin'] = item['Cabin'].replace(['F','G'], 0)
    item['Cabin'] = item['Cabin'].replace(['N'], 1)

In [ ]:
 train[['Cabin','Survived']].groupby(['Cabin'], as_index=False).mean()

In [ ]:
train['Embarked'] = train['Embarked'].fillna('S')
combo = [train, test]
for item in combo:
    item['Embarked'] = item['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [ ]:
combo = [train, test]
for item in combo:
# We replace missing ages by the mean age of passengers who belong to the same group of class/sex/family
 item['Age'] = item.groupby(['Pclass','Title'])['Age'].apply(lambda x: x.fillna(x.mean()))

In [ ]:
#complete missing fare with median
test['Fare'].fillna(test['Fare'].median(), inplace = True)

In [ ]:
train['FareBin'] = pd.qcut(train['Fare'], 10)
train[['FareBin','Survived']].groupby(['FareBin'], as_index=False).mean()

In [ ]:
for item in combo:
    item.loc[ item['Fare'] <= 7.55, 'Fare'] = 0
    item.loc[(item['Fare'] > 7.55) & (item['Fare'] <= 10.5), 'Fare'] = 1
    item.loc[(item['Fare'] > 10.5) & (item['Fare'] <= 14.454), 'Fare'] = 2
    item.loc[(item['Fare'] > 14.454) & (item['Fare'] <= 21.67), 'Fare']   = 3
    item.loc[(item['Fare'] > 21.67) & (item['Fare'] <= 77.958), 'Fare']   = 4
    item.loc[ item['Fare'] > 77.958, 'Fare'] = 5
    item['Fare'] = item['Fare'].astype(int)    

In [ ]:
 train[['Pclass','Survived']].groupby(['Pclass'], as_index=False).mean()

In [ ]:
 train[['Fare','Survived']].groupby(['Fare'], as_index=False).mean()

In [ ]:
for item in combo:
   item = item.drop(['Ticket'], axis=1, inplace=True)

In [ ]:
# Explore Age vs Survived
g = sns.FacetGrid(train, col='Survived', height=4, aspect=1, sharex='none')
g = g.map(sns.distplot, "Age", bins=20)

In [ ]:
train['Age2']=-1
test['Age2']=-1

combo=[train,test]
for item in combo:
    item.loc[(item['Age'] <=4), 'Age2'] =0
    item.loc[(item['Age']<=17 )& (item['Age'] >4), 'Age2'] =1
    item.loc[(item['Age'] >=18) & (item['Age'] <=32), 'Age2'] =2
    item.loc[(item['Age'] >32) & (item['Age'] <=42), 'Age2']= 3
    item.loc[(item['Age'] >42) & (item['Age'] <=52), 'Age2']= 4
    item.loc[(item['Age'] >52) & (item['Age'] <=60), 'Age2']= 5
    item.loc[(item['Age'] >60) , 'Age2']= 6

In [ ]:
 train[['Survived','Age2']].groupby(['Age2'], as_index=False).mean()

In [ ]:
sns.barplot( x="Age2",y="Survived", data=train)
plt.show()

In [ ]:
import scipy.stats
from scipy.stats import pearsonr


pearson_coef1 , p_value1 =scipy.stats.pearsonr(train["Cabin"],train["Pclass"])
print ("Cabin - Pclass", pearson_coef1,"    Correlation certainty =",p_value1)

In [ ]:
sns.factorplot('Pclass','Survived',hue='Sex',data=train)
plt.show()

In [ ]:
for item in combo:
    item = item.drop(['Sex'], axis=1, inplace=True)

In [ ]:
for item in combo:
    item.loc[((item['Title']=='Miss') & (item['Age'] <=4)), 'Title'] = 'child'
    item.loc[((item['Title']=='Master') & (item['Age'] <=4)), 'Title'] = 'child'
    item.loc[((item['Title']=='Miss') & (item['Age'] <=17)), 'Title'] = 'kid'
    item.loc[((item['Title']=='Master') & (item['Age'] <=17)), 'Title'] = 'kid'
    item.loc[((item['Title']=='Master') & (item['Age'] >17)), 'Title'] = 'Mr'
    item.loc[((item['Title']=='Mrs') & (item['Parch']!=0)),'Title']='MrsYesCh'
    item.loc[((item['Title']=='Mrs') & (item['Parch']==0)),'Title']='MrsNoCh'
    item = item.drop(['SibSp','Parch'], axis=1, inplace=True)

In [ ]:
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
train['Title'] = train['Title'].map({ 'Mr': 0 ,'Miss':1, 'MrsNoCh':2 ,'MrsYesCh':3,'kid':4,'child':5}).astype(int)
test['Title'] = test['Title'].map({'Mr': 0 ,'Miss':1, 'MrsNoCh':2 ,'MrsYesCh':3,'kid':4, 'child':5}).astype(int)

In [ ]:
sns.barplot( x="Embarked",y="Survived", data=train)
plt.show()

In [ ]:
for item in combo:
    item = item.drop(['Age'], axis=1, inplace=True)

In [ ]:
#set up the data for the models
X_train = train.drop(['Survived','PassengerId','FamilySize','FareBin'], axis=1)
col=(X_train.columns)
Y_train = train["Survived"]
X_test=test.drop(["PassengerId","FamilySize"], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()  
scaler.fit_transform(X_train)
scaler.transform(X_test)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif
from numpy import set_printoptions

test1 = SelectKBest(score_func= f_classif, k=4)
fit = test1.fit(X_train, Y_train)
set_printoptions(precision=3)
print(col)
print('       ',fit.scores_)

The top three most important features are Title, Pclass and Fare 

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
# feature extraction
model = ExtraTreesClassifier(n_estimators=10)
model.fit(X_train, Y_train)
print(col)
print('Feature importance')
print(model.feature_importances_)

In [ ]:
import xgboost as xgb
import shap 
shap.initjs() 
dmatrix=xgb.DMatrix(X_train, label=Y_train)
model = xgb.train({"learning_rate": 0.01}, dmatrix, 100)
model_bytearray = model.save_raw()[4:]

def myfun(self=None):
    return model_bytearray

model.save_raw = myfun
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_train, plot_type="bar")


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=4)
fit = rfe.fit(X_train, Y_train)
print('Features :  ',col)
print("Num Features: ",fit.n_features_)
print("Selected Features: ",fit.support_)
print("Feature Ranking: ",fit.ranking_)

In [ ]:
X_train = pd.get_dummies(X_train, columns = ["Title"],prefix="N")
X_train = pd.get_dummies(X_train, columns = ["Fare"],prefix="F",drop_first=True)
X_test = pd.get_dummies(X_test, columns = ["Title"],prefix="N")
X_test = pd.get_dummies(X_test, columns = ["Fare"],prefix="F",drop_first=True)
Y_train = train["Survived"].values

In [ ]:
X_train.drop(["Embarked","Cabin",'Age2'], axis=1, inplace=True)

X_test.drop(["Embarked","Cabin",'Age2'], axis=1,inplace=True)

print("Training set X: ", X_train.shape, "Training set Y:",Y_train.shape, "Testing set X", X_test.shape)

In [ ]:
sns.heatmap(X_train.corr(), cmap='viridis',annot = True)
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

import keras
from keras import models,layers,optimizers
from keras.optimizers import Adam 
from keras.layers import Activation, Dense ,Dropout 
from keras.callbacks import EarlyStopping

from numpy import mean , std
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from numpy import arange, argmax
from sklearn import metrics

from sklearn.metrics import auc,confusion_matrix,classification_report,roc_curve,accuracy_score,roc_auc_score



In [ ]:
randomforest = RandomForestClassifier(criterion = 'gini',bootstrap= True, max_depth= 5, max_features=10,  min_samples_split= 31, n_estimators=20,random_state=seed_value)
svc_model = SVC(C=0.5, cache_size=200, 
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf', max_iter=-1, probability=True, random_state=seed_value, shrinking=True, tol=0.001,verbose=False)
knn = KNeighborsClassifier(leaf_size= 6,n_neighbors = 16, p=1 ,weights = 'uniform') 
modellt =  LGBMClassifier(boosting_type='gbdt',num_leaves = 31,max_depth =5,learning_rate =0.05,subsample= 1,colsample_bytree = 0.8, reg_lambda =0.5,reg_alpha=                          0.5,
                          n_estimators=500,objective='binary',metric='binary_logloss', seed=seed_value)
#check xgb
xgb_model=XGBClassifier( alpha =0.5, colsample_bytree =0.8 ,gamma=0.5,learning_rate= 0.1, max_depth= 5,n_estimators=100,
                         objective= 'binary:logistic',subsample= 0.8, random_state=seed_value)
modelM=MLPClassifier(hidden_layer_sizes=(16,8,1),activation='tanh',solver='adam', alpha=0.01, batch_size=32, learning_rate='adaptive', learning_rate_init=0.001, max_iter=850, shuffle=False, random_state=seed_value, tol=0.001, verbose=False,early_stopping=False,validation_fraction=0.3, beta_1=0.9, beta_2=0.999, epsilon=1e-08 )
AdaB=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),learning_rate=0.001,n_estimators=100,random_state=seed_value)
gb = GradientBoostingClassifier(criterion='mae',n_estimators=150,learning_rate = 0.1,max_features=7,max_depth =3, 
            min_samples_leaf=3, min_samples_split= 32, subsample= 1,random_state=seed_value)


NN1=('NN', modelM)
XG1=('XGboost', xgb_model)
GR1=('GradientB',gb)
SVM1= ('SVM',svc_model)
ADA1=('Ada',AdaB)
KNN1=('KNN',knn)
RF1=('RF',randomforest)
LG1=('LightGBM',modellt)

In [ ]:
def evaluate_model(model):
    seed=7
    kfld = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
    scores = cross_val_score(model, X_train , Y_train, scoring='accuracy', cv=kfld, n_jobs=-1, error_score='raise')
    return scores

clfs={'LightGBM':[modellt,78.707],'XGBoost':[xgb_model,99978.229],'GradientBoost':[gb,78.468]
            ,'AdaBoost':[AdaB,78.229],'RandomForest':[randomforest,78.707],
            'SVM':[svc_model,78.229],'KNN':[knn,80.143],'MLP':[modelM,79.168]}
scores={}
for name, model in clfs.items():
    metr=evaluate_model(model[0])
    scores.update({name:[mean(metr),std(metr),model[1]]})

results_table = pd.DataFrame(scores, index=['Mean_accuracy', 'Std', 'Leaderboard']).round(decimals=3)
results_table

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y):
 
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y , cv=10, n_jobs=-1,train_sizes=np.linspace(.1, 1.0, 5),random_state = 2509)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.plot(train_sizes, train_scores_mean, 'o-', color="b", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="orange",  label="Cross-validation score")
    
    # Plot learning curve
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,color="b")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1, color="orange")
    
models_set={'LightGBM':modellt,'XGBoost':xgb_model,'GradientBoost':gb,'AdaBoost':AdaB,
            'RandomForest':randomforest,'SVM':svc_model,'KNN':knn,'MLP':modelM}

ax=0
plt.figure(figsize = (25,25))
plt.suptitle("Models Learning Curves", fontsize = 28)

for name, model in models_set.items():
     ax+=1
     plt.subplot(4,2,ax)
     plt.title(name, fontsize = 18)
     plot_learning_curve(model,name,X_train,Y_train)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2,shuffle=True ,random_state=123)

In [ ]:
from sklearn.metrics import plot_roc_curve

classifiers = [knn,randomforest, modelM, modellt,xgb_model,gb,svc_model,AdaB]
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
for clf in classifiers:
     model = clf.fit(x_train, y_train)
     plot_roc_curve(model, x_test, y_test, ax=ax)
 

In [ ]:
from sklearn.metrics import balanced_accuracy_score,f1_score,average_precision_score
def evaluation(name,clf):
    model=clf.fit(x_train,y_train)
    y_prob = model.predict_proba(x_test)[:, 1]
    prAuc = average_precision_score(y_test, y_prob)
    y_pred = model.predict(x_test)
    acc = balanced_accuracy_score(y_test, y_pred)
    f1= f1_score(y_test, y_pred)
    return prAuc,acc,f1     
 
scores={}
for name, model in models_set.items():
    metr=evaluation(name,model)
    scores.update({name:[metr[0],metr[1],metr[2]]})
results = (pd.DataFrame(scores, index=['PR-AUC','Balanced_Accuracy','F1 score'])).round(decimals=3)
results

In [ ]:
def clf_predictions(model):
    model.fit(X_train, Y_train)
    ypredict = model.predict(X_test)
    return ypredict

predictions = {name: clf_predictions(model) for name, model in models_set.items()}
estimations = pd.DataFrame(predictions) 
sns.heatmap(estimations.corr(),cmap='viridis', annot = True)
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

In [ ]:
# function to create classifiers using Voting or Stacking
def create_clf(E,t):
    if t=='Voting':    
        vclf=VotingClassifier(E, voting='hard')
    else:
        vclf=StackingClassifier(E, final_estimator= LogisticRegression())
    return vclf

#Step1 
E1=[KNN1,NN1,SVM1,RF1,GR1]
#Step2 
E2=[KNN1,SVM1,RF1,GR1]
#Step3 
E3=[KNN1,SVM1,GR1] 
#Step4 
E4=[KNN1,SVM1]

ensembles_set={'KNN-NN-SVM-RF-GR':[E1,'79.186','78.947'],'KNN-SVM-RF-GR':[E2,'79.425','79.186 ']
             ,'KNN-SVM-GR':[E3,'79.425','79.186'],'KNN-SVM1':[E4,'80.382','78.468']}
scores1={}
scores2={}
for clf, item in ensembles_set.items():
    newclv=create_clf(item[0],'Voting')
    newclST=create_clf(item[0],'Stacking')
    kfoldevalv=evaluate_model(newclv)
    kfoldevalst=evaluate_model(newclST)
    scores1.update({clf:[mean(kfoldevalv),std(kfoldevalv),item[1]]})
    scores2.update({clf:[mean(kfoldevalst),std(kfoldevalst),item[2]]})
results1 = pd.DataFrame(scores1, index=['Mean_accuracy', 'Std','Leaderboard'])
results2 = pd.DataFrame(scores2, index=['Mean_accuracy', 'Std','Leaderboard'])
print('VOTING CLASSIFIERS')
display(results1)
print('STACKING CLASSIFIERS')
display(results2)

In [ ]:
E=[KNN1,SVM1]
vot=VotingClassifier(E, voting='hard')
ensemble = vot.fit(X_train, Y_train) 
y_vote = ensemble.predict(X_test)

submission = pd.DataFrame({
            "PassengerId": test["PassengerId"],
            "Survived": y_vote
    })
submission.to_csv('Titanic.csv', index=False)
print("Your submission was successfully saved!") 
